In [37]:
import argparse
import pickle
import logging
from pathlib import Path
from typing import Dict, Any, Optional, List, Tuple
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack, csr_matrix
from xgboost import XGBClassifier
import pandas as pd

In [36]:
from xgboost import XGBClassifier

In [ ]:
def page_relevance_analysis(df, y_proba, best_threshold):
    df_adjusted = df.copy()
    df_adjusted['relevance_probability'] = y_proba
    
    
    
    
    df_adjusted['threshold'] = best_threshold
    df_adjusted['relevance_decision'] = (y_proba >= best_threshold).astype(int)
    
    
    
    
    

In [44]:
test_size = 0.3
random_state = 42
oversample_positive = True
unigram_max_features = 5000
trigram_max_features = 2000
c_regularization = 1.0
max_iter = 1000

# Load and clean data
df = pd.read_csv("pdf_classifications.csv")
df.dropna(inplace=True)
df = df[(df['keep'] == '0') | (df['keep'] == '1')]
df['keep'] = df['keep'].astype(int)
df['text'] = df['text'].str.replace('\n', ' ')

print(f"Training on {len(df)} pages")
print(f"Class distribution: {df['keep'].value_counts().to_dict()}")

X = df['text'].fillna('').astype(str).values
y = df['keep'].values

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, random_state=random_state, stratify=y
)

# Oversample positive class
def oversample_positive_pages(texts, labels, random_state):
    """Oversample positive class to match negatives in the training split."""
    positives = np.where(labels == 1)[0]
    negatives = np.where(labels == 0)[0]
    
    if len(positives) == 0 or len(negatives) == 0:
        print("Warning: Skipping oversampling due to missing class in training split.")
        return texts, labels
    
    if len(positives) >= len(negatives):
        return texts, labels
    
    rng = np.random.default_rng(seed=random_state)
    sampled_pos = rng.choice(positives, size=len(negatives), replace=True)
    new_indices = np.concatenate([negatives, sampled_pos])
    rng.shuffle(new_indices)
    
    return texts[new_indices], labels[new_indices]

if oversample_positive:
    X_train, y_train = oversample_positive_pages(X_train, y_train, random_state)
    print(f"Applied positive-class oversampling. New train distribution: "
          f"{{0: {int((y_train == 0).sum())}, 1: {int((y_train == 1).sum())}}}")

# Extract TF-IDF statistics
def extract_tfidf_statistics(texts, tfidf_matrix):
    """Extract TF-IDF-based statistical features from text."""
    features = []
    
    for i, text in enumerate(texts):
        text_lower = text.lower()
        text_words = text_lower.split()
        
        # Get TF-IDF vector for this document
        doc_tfidf = tfidf_matrix[i].toarray().flatten()
        
        # Statistical features from TF-IDF
        stats = [
            np.sum(doc_tfidf),
            np.mean(doc_tfidf),
            np.std(doc_tfidf),
            np.max(doc_tfidf),
            np.percentile(doc_tfidf, 75),
            np.percentile(doc_tfidf, 50),
            np.percentile(doc_tfidf, 25),
            np.sum(doc_tfidf > 0),
            len(text_words),
            len(text) / max(len(text_words), 1),
        ]
        
        # Additional text-based features
        stats.extend([
            text.count('\n'),
            text.count('.'),
            text.count('='),
            text.count('('),
            text.count('['),
            len([w for w in text_words if len(w) > 10]),
        ])
        
        features.append(stats)
    
    return csr_matrix(features)

# Create TF-IDF vectorizers
vectorizer = TfidfVectorizer(
    max_features=unigram_max_features,
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.95,
    stop_words='english',
    sublinear_tf=True,
    norm='l2'
)

trigram_vectorizer = TfidfVectorizer(
    max_features=trigram_max_features,
    ngram_range=(3, 3),
    min_df=3,
    max_df=0.9,
    stop_words='english',
    sublinear_tf=True,
    norm='l2'
)

# Fit vectorizers and transform training data
print("Extracting TF-IDF features...")
X_train_tfidf = vectorizer.fit_transform(X_train)
X_train_trigram = trigram_vectorizer.fit_transform(X_train)
X_train_stats = extract_tfidf_statistics(X_train, X_train_tfidf)
X_train_combined = hstack([X_train_tfidf, X_train_trigram, X_train_stats])

# Transform test data
X_test_tfidf = vectorizer.transform(X_test)
X_test_trigram = trigram_vectorizer.transform(X_test)
X_test_stats = extract_tfidf_statistics(X_test, X_test_tfidf)
X_test_combined = hstack([X_test_tfidf, X_test_trigram, X_test_stats])

# Normalize statistical features
scaler = StandardScaler()
n_stats = X_train_stats.shape[1]
stats_start_idx = X_train_combined.shape[1] - n_stats

X_train_stats_dense = X_train_combined[:, stats_start_idx:].toarray()
X_test_stats_dense = X_test_combined[:, stats_start_idx:].toarray()

X_train_stats_scaled = scaler.fit_transform(X_train_stats_dense)
X_test_stats_scaled = scaler.transform(X_test_stats_dense)

# Recombine with scaled stats
X_train_final = hstack([
    X_train_combined[:, :stats_start_idx],
    csr_matrix(X_train_stats_scaled)
])
X_test_final = hstack([
    X_test_combined[:, :stats_start_idx],
    csr_matrix(X_test_stats_scaled)
])

# Train logistic regression classifier
print(f"Training with C={c_regularization}, max_iter={max_iter}")
print("Using 'balanced' class weights")

classifier = LogisticRegression(
    class_weight={0: 1.0, 1: 5.0},
    max_iter=max_iter,
    random_state=random_state,
    solver='lbfgs',
    C=c_regularization
)

classifier = XGBClassifier(
    scale_pos_weight=3.0,  # Penalize false negatives
    max_depth=6,
    learning_rate=0.1,
    n_estimators=200,
    random_state=random_state,
    use_label_encoder=False,
    eval_metric='logloss'
)

print("Training classifier...")
classifier.fit(X_train_final, y_train)

# Evaluate with default threshold (0.5)
y_pred = classifier.predict(X_test_final)
y_proba = classifier.predict_proba(X_test_final)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
print(f"\nTest accuracy: {accuracy:.4f}")
print("\nClassification Report (threshold=0.5):")
print(classification_report(y_test, y_pred, target_names=['Not Relevant', 'Relevant']))

# Find optimal threshold
thresholds = np.arange(0.3, 0.8, 0.05)
best_f1 = 0
best_threshold = 0.5
best_precision = 0
best_recall = 0

for threshold in thresholds:
    y_pred_thresh = (y_proba >= threshold).astype(int)
    f1 = f1_score(y_test, y_pred_thresh, pos_label=1, zero_division=0)
    precision = precision_score(y_test, y_pred_thresh, pos_label=1, zero_division=0)
    recall = recall_score(y_test, y_pred_thresh, pos_label=1, zero_division=0)
    
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold
        best_precision = precision
        best_recall = recall

print(f"\n📊 Optimal threshold for 'Relevant' class: {best_threshold:.2f}")
print(f"   Precision: {best_precision:.3f}, Recall: {best_recall:.3f}, F1: {best_f1:.3f}")

# Show metrics at different thresholds
print("\n📈 Metrics at different thresholds:")
for thresh in [0.4, 0.5, 0.6, 0.7]:
    y_pred_thresh = (y_proba >= thresh).astype(int)
    prec = precision_score(y_test, y_pred_thresh, pos_label=1, zero_division=0)
    rec = recall_score(y_test, y_pred_thresh, pos_label=1, zero_division=0)
    f1 = f1_score(y_test, y_pred_thresh, pos_label=1, zero_division=0)
    print(f"   Threshold {thresh:.1f}: Precision={prec:.3f}, Recall={rec:.3f}, F1={f1:.3f}")

# Save model
model_data = {
    'vectorizer': vectorizer,
    'trigram_vectorizer': trigram_vectorizer,
    'scaler': scaler,
    'classifier': classifier
}

#joblib.dump(model_data, 'pdf_classifier.pkl')
#print(f"\nModel saved to pdf_classifier.pkl")

Training on 1601 pages
Class distribution: {0: 1322, 1: 279}
Applied positive-class oversampling. New train distribution: {0: 925, 1: 925}
Extracting TF-IDF features...
Training with C=1.0, max_iter=1000
Using 'balanced' class weights
Training classifier...


/Users/tarunshah/Desktop/Venv VS/Annapurna Fall 2025/WebScraping Engine/fa25-annapurna/.venv/lib/python3.13/site-packages/xgboost/training.py:199: UserWarning: [17:51:15] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



Test accuracy: 0.8482

Classification Report (threshold=0.5):
              precision    recall  f1-score   support

Not Relevant       0.89      0.94      0.91       397
    Relevant       0.59      0.43      0.50        84

    accuracy                           0.85       481
   macro avg       0.74      0.68      0.70       481
weighted avg       0.83      0.85      0.84       481


📊 Optimal threshold for 'Relevant' class: 0.30
   Precision: 0.523, Recall: 0.536, F1: 0.529

📈 Metrics at different thresholds:
   Threshold 0.4: Precision=0.563, Recall=0.476, F1=0.516
   Threshold 0.5: Precision=0.590, Recall=0.429, F1=0.497
   Threshold 0.6: Precision=0.623, Recall=0.393, F1=0.482
   Threshold 0.7: Precision=0.674, Recall=0.345, F1=0.457


In [ ]:
df['keep'].value_counts()

keep
0      1322
1       279
]0        3
0 0       1
o         1
Name: count, dtype: int64

In [ ]:
df = df[(df['keep'] == '0') | (df['keep'] == '1')]

In [ ]:
df

,paper_id,pdf_name,page_number,text,keep
0,9ad32df4723703816a30c7c6995b646ed4779f0bbad499...,Diffusion_Bridge_Implicit_Models.pdf,1,Published as a conference paper at ICLR 2025\n...,0
1,9ad32df4723703816a30c7c6995b646ed4779f0bbad499...,Diffusion_Bridge_Implicit_Models.pdf,2,Published as a conference paper at ICLR 2025\n...,1
2,9ad32df4723703816a30c7c6995b646ed4779f0bbad499...,Diffusion_Bridge_Implicit_Models.pdf,3,Published as a conference paper at ICLR 2025\n...,1
3,9ad32df4723703816a30c7c6995b646ed4779f0bbad499...,Diffusion_Bridge_Implicit_Models.pdf,4,Published as a conference paper at ICLR 2025\n...,1
4,9ad32df4723703816a30c7c6995b646ed4779f0bbad499...,Diffusion_Bridge_Implicit_Models.pdf,5,Published as a conference paper at ICLR 2025\n...,1
...,...,...,...,...,...
1604,61552bb2cd5d7c1e879a0cf1d4461834daa30b4c506f69...,SpinQuant__LLM_Quantization_with_Learned_Rotat...,21,Published as a conference paper at ICLR 2025\n...,0
1605,61552bb2cd5d7c1e879a0cf1d4461834daa30b4c506f69...,SpinQuant__LLM_Quantization_with_Learned_Rotat...,22,Published as a conference paper at ICLR 2025\n...,0
1606,61552bb2cd5d7c1e879a0cf1d4461834daa30b4c506f69...,SpinQuant__LLM_Quantization_with_Learned_Rotat...,23,Published as a conference paper at ICLR 2025\n...,0
1607,61552bb2cd5d7c1e879a0cf1d4461834daa30b4c506f69...,SpinQuant__LLM_Quantization_with_Learned_Rotat...,24,Published as a conference paper at ICLR 2025\n...,0


In [31]:
X_train

array(['Published as a conference paper at ICLR 2025 Here, we additionally use a scheduler κ(t), that satisfies κ(0) = 1 and κ(1) = 0, to control the rate at which the geodesic distance d between x0 and x1 decreases (Chen & Lipman, 2023): d(xt, x1) = κ(t)d(x0, x1). (15) Thus, we obtain the loss function Lχ(θ) = Et,q(x1),p(x0)∥vθ(xt, t) −˙xt∥2 (16) with ˙xt = −˙κ(t) logx0(x1). (17) Here, the learned vector field vθ(xt, t) represents vectors on the tangent plane. A.1.3 MARKOV BRIDGE MODEL The molecular graph consists of discrete entities (node and edge types) and can therefore not be easily modeled in the flow matching framework. While discrete diffusion formulations (Austin et al., 2021; Vignac et al., 2022) can be used in principle, we decided to employ the Markov bridge model (Igashov et al., 2023) instead which is conceptually more similar to the flow matching scheme used for the continuous variables as it does not require a closed-form prior. The Markov bridge model captures the sto

In [42]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, classification_report, f1_score, 
    precision_score, recall_score
)
from sentence_transformers import SentenceTransformer
import joblib

# Configuration
test_size = 0.3
random_state = 42
oversample_positive = False
c_regularization = 1.0
max_iter = 1000
embedding_model_name = 'all-MiniLM-L6-v2'  # Fast and efficient
# Alternative: 'all-mpnet-base-v2' for better quality but slower

# Load and clean data
df = pd.read_csv("pdf_classifications.csv")
df.dropna(inplace=True)
df = df[(df['keep'] == '0') | (df['keep'] == '1')]
df['keep'] = df['keep'].astype(int)
df['text'] = df['text'].str.replace('\n', ' ')

print(f"Training on {len(df)} pages")
print(f"Class distribution: {df['keep'].value_counts().to_dict()}")

X = df['text'].astype(str).values
y = df['keep'].values

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, random_state=random_state, stratify=y
)

# Oversample positive class
def oversample_positive_pages(texts, labels, random_state):
    """Oversample positive class to match negatives in the training split."""
    positives = np.where(labels == 1)[0]
    negatives = np.where(labels == 0)[0]
    
    if len(positives) == 0 or len(negatives) == 0:
        print("Warning: Skipping oversampling due to missing class in training split.")
        return texts, labels
    
    if len(positives) >= len(negatives):
        return texts, labels
    
    rng = np.random.default_rng(seed=random_state)
    sampled_pos = rng.choice(positives, size=len(negatives), replace=True)
    new_indices = np.concatenate([negatives, sampled_pos])
    rng.shuffle(new_indices)
    
    return texts[new_indices], labels[new_indices]

if oversample_positive:
    X_train, y_train = oversample_positive_pages(X_train, y_train, random_state)
    print(f"Applied positive-class oversampling. New train distribution: "
          f"{{0: {int((y_train == 0).sum())}, 1: {int((y_train == 1).sum())}}}")

# Load transformer model and generate embeddings
print(f"\nLoading transformer model: {embedding_model_name}")
print("This may take a moment on first run (downloads model)...")
embedding_model = SentenceTransformer(embedding_model_name)

print("\nGenerating embeddings for training data...")
X_train_embeddings = embedding_model.encode(
    X_train, 
    show_progress_bar=True,
    batch_size=32,
    convert_to_numpy=True
)

print("Generating embeddings for test data...")
X_test_embeddings = embedding_model.encode(
    X_test,
    show_progress_bar=True,
    batch_size=32,
    convert_to_numpy=True
)

print(f"Embedding shape: {X_train_embeddings.shape}")

# Train logistic regression classifier with custom class weights
print(f"\nTraining with C={c_regularization}, max_iter={max_iter}")
print("Using custom class weights: {0: 1.0, 1: 5.0} to penalize false negatives")

classifier = LogisticRegression(
    class_weight={0: 1.0, 1: 5.0},  # Penalize false negatives 3x more
    max_iter=max_iter,
    random_state=random_state,
    solver='lbfgs',
    C=c_regularization
)

print("Training classifier...")
classifier.fit(X_train_embeddings, y_train)

# Evaluate with default threshold (0.5)
y_pred = classifier.predict(X_test_embeddings)
y_proba = classifier.predict_proba(X_test_embeddings)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
print(f"\nTest accuracy: {accuracy:.4f}")
print("\nClassification Report (threshold=0.5):")
print(classification_report(y_test, y_pred, target_names=['Not Relevant', 'Relevant']))

# Find optimal threshold
thresholds = np.arange(0.3, 0.8, 0.05)
best_f1 = 0
best_threshold = 0.5
best_precision = 0
best_recall = 0

for threshold in thresholds:
    y_pred_thresh = (y_proba >= threshold).astype(int)
    f1 = f1_score(y_test, y_pred_thresh, pos_label=1, zero_division=0)
    precision = precision_score(y_test, y_pred_thresh, pos_label=1, zero_division=0)
    recall = recall_score(y_test, y_pred_thresh, pos_label=1, zero_division=0)
    
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold
        best_precision = precision
        best_recall = recall

print(f"\n📊 Optimal threshold for 'Relevant' class: {best_threshold:.2f}")
print(f"   Precision: {best_precision:.3f}, Recall: {best_recall:.3f}, F1: {best_f1:.3f}")

# Show metrics at different thresholds
print("\n📈 Metrics at different thresholds:")
for thresh in thresholds:
    y_pred_thresh = (y_proba >= thresh).astype(int)
    prec = precision_score(y_test, y_pred_thresh, pos_label=1, zero_division=0)
    rec = recall_score(y_test, y_pred_thresh, pos_label=1, zero_division=0)
    f1 = f1_score(y_test, y_pred_thresh, pos_label=1, zero_division=0)
    print(f"   Threshold {thresh:.1f}: Precision={prec:.3f}, Recall={rec:.3f}, F1={f1:.3f}")

# Calculate and display confusion matrix at optimal threshold
from sklearn.metrics import confusion_matrix

y_pred_optimal = (y_proba >= best_threshold).astype(int)
cm = confusion_matrix(y_test, y_pred_optimal)
tn, fp, fn, tp = cm.ravel()

print(f"\n📋 Confusion Matrix at optimal threshold ({best_threshold:.2f}):")
print(f"   True Negatives:  {tn:4d}  |  False Positives: {fp:4d}")
print(f"   False Negatives: {fn:4d}  |  True Positives:  {tp:4d}")
print(f"\n   False Negative Rate: {fn/(fn+tp):.3f} (proportion of relevant pages missed)")
print(f"   False Positive Rate: {fp/(fp+tn):.3f} (proportion of irrelevant pages included)")

# Save model
model_data = {
    'embedding_model_name': embedding_model_name,
    'embedding_model': embedding_model,
    'classifier': classifier,
    'optimal_threshold': best_threshold
}

# Uncomment to save:
# joblib.dump(model_data, 'pdf_classifier_transformer.pkl')
# print(f"\nModel saved to pdf_classifier_transformer.pkl")

print("\n✅ Training complete!")
print(f"\nTo use this model for predictions:")
print(f"1. Load the model: model_data = joblib.load('pdf_classifier_transformer.pkl')")
print(f"2. Generate embeddings: embeddings = model_data['embedding_model'].encode(new_texts)")
print(f"3. Get predictions: probs = model_data['classifier'].predict_proba(embeddings)[:, 1]")
print(f"4. Apply threshold: predictions = (probs >= {best_threshold:.2f}).astype(int)")

Training on 1601 pages
Class distribution: {0: 1322, 1: 279}

Loading transformer model: all-MiniLM-L6-v2
This may take a moment on first run (downloads model)...

Generating embeddings for training data...


Batches: 100%|██████████| 35/35 [00:02<00:00, 12.32it/s]


Generating embeddings for test data...


Batches: 100%|██████████| 16/16 [00:01<00:00, 13.49it/s]

Embedding shape: (1120, 384)

Training with C=1.0, max_iter=1000
Using custom class weights: {0: 1.0, 1: 5.0} to penalize false negatives
Training classifier...

Test accuracy: 0.7152

Classification Report (threshold=0.5):
              precision    recall  f1-score   support

Not Relevant       0.89      0.74      0.81       397
    Relevant       0.32      0.58      0.42        84

    accuracy                           0.72       481
   macro avg       0.61      0.66      0.61       481
weighted avg       0.79      0.72      0.74       481


📊 Optimal threshold for 'Relevant' class: 0.45
   Precision: 0.304, Recall: 0.690, F1: 0.422

📈 Metrics at different thresholds:
   Threshold 0.3: Precision=0.245, Recall=0.893, F1=0.385
   Threshold 0.3: Precision=0.269, Recall=0.857, F1=0.409
   Threshold 0.4: Precision=0.290, Recall=0.750, F1=0.419
   Threshold 0.4: Precision=0.304, Recall=0.690, F1=0.422
   Threshold 0.5: Precision=0.325, Recall=0.583, F1=0.417
   Threshold 0.5: Precision=0